# Part C. Graph Algorithms

## 1. Introduction

For the last task in this lab assignment we use one of the presented algorithms to inspect the data in a different way in a strive to learn new aspects of it. The algorithm we uplan to use in this section is a Centrality Algorithm, that will measure our most influential authors. For this, we will use PageRank.

PageRank is a website ranking algorithm that asks: **which pages are the most “important”?**

Pages that have many incoming links — especially from other well‑linked pages — are more central to the Website network, thus more influential. That centrality becomes the PageRank score. 

The idea in this section is to adapt the same idea to our citation network to find influential authors instead of pages.

These are the numbers that our dataset has, which allow us to work nicely with this algorithm:

- Total Authors =3866419

- Total Articles = 3837322

- Total Author → Article edges = 13132253



After an initial inspection of the data we stumble upon what could become a problem. We find that the vast majority of articles have a signle author, but the articles with the highest amount of authors have very disproportionate numbers, which could damage our results. Below are the queries used for this small inspection: 

### Query to randomise a sample of 50 journals and collect their respective number of authors. 

```bash
MATCH (p:article)-[:authored_by]->(a:author)
WITH p, count(a) AS authorCount, rand() AS r
ORDER BY r
LIMIT 50
RETURN
  p.key, p.title, authorCount;

### Query to extract the number of authors the top 70 papers with the highest amounts of authors have

```bash
MATCH (p:article)-[:authored_by]->(a:author)
WITH p, count(a) AS authorCount
RETURN
  p.key           AS articleKey,
  p.title         AS articleTitle,
  authorCount     AS numAuthors
ORDER BY numAuthors DESC
LIMIT 70;

### Query to count how many articles have over 5 authors

```bash
MATCH 
  (a:author)<-[:authored_by]-(p:article)
WITH 
  p, 
  count(DISTINCT a) AS numAuthors
WHERE 
  numAuthors > 5
RETURN 
  count(DISTINCT p) AS journalsWithOver5Authors;

## 2. Preprocessing

From this very last query, we can observe that, out of 3837322 articles, we have 491979 with over 5 authors, 39161 with over 10, and only 4540 with 20 authors or more. The top 70 ariticles with hgihest amount of authors all have over 100 authors (some with more than 400), which can be a threat to our algorithm. Let's explore why:

A paper with N authors that cites another paper with M authors creates N×M author→author edges. for large N or M, we can easily add tens of thousands of edges, from a single publication. 

Additionally, all those edges carry equal weight in classic PageRank. An author on a 200‑author paper “votes” just as strongly as a sole author on a 1‑author paper. This inflates the connectivity of authors who simply happened to join large consortia. PageRank rewards nodes with high in‑degree and connections to other high‑rank nodes, hence A few mega‑author papers can  dominate the rank distribution, concealing the truly more influential authors. 

To avoid this problems, we slightly switch our strategy. Instead of measuring the most influential authors, we will only measure the most influential papers. This means that our nodes will now be articles, with citations amongst each other. 

## 3. Algorithm and query definition

Let's now go over the pipeline:
We firstly create a citation graph. We want a directed graph where each node is an article, and there’s an edge
A → B whenever any paper  A is cited by any paper B. 

We then run the PageRank algorithm, and finally inspect the results. Note that we use a damping effect of 0.85, which avoids rank sinks ( strands of uncited papers). This algorithm allows us to not only highlight papers that are not just heavily cited, but whose citers are themselves influential.

Due to the lack of citation data we have commented on before, we have automated the creation of some citation relationships. For a total sample of around 150 articles, we have created one or two citations to other articles. this allows us to see a mini version of Page Rank, ramking among that subgraph.


 Below are the Cypher queries to do so:

```bash
// 1. Drop any existing projection with that name
CALL gds.graph.drop('articleCiteGraph', false)
YIELD graphName;

// 2. Project a new in‑RAM graph called “articleCiteGraph”
//    - Nodes: all nodes with the label :article
//    - Rels :HAS_CITATION, directed
CALL gds.graph.project(
  'articleCiteGraph',           // name of your in‑RAM graph
  ['article'],                  // list of node labels to include
  {HAS_CITATION: {type: 'HAS_CITATION'}}  // map of rel‑types to include
)
YIELD graphName, nodeCount, relationshipCount;

RETURN graphName, nodeCount, relationshipCount;



// part 2. Run PageRank Algorithm
CALL gds.pageRank.stream(
  'articleCiteGraph',
  {
    maxIterations: 20,
    dampingFactor: 0.85
  }
)
YIELD nodeId, score
RETURN
  gds.util.asNode(nodeId).title  AS paperTitle,
  gds.util.asNode(nodeId).key    AS paperKey,
  round(score,4)                 AS pageRankScore
ORDER BY pageRankScore DESC
LIMIT 10;


The previous algorithm is implemented in seconds and yields the following results

## 4. Results

In [4]:
import pandas as pd
df=pd.read_csv('PageRank.csv')
df.head(10)

,paperTitle,paperKey,pageRankScore
0,Entropy-accelerated exact clustering of protei...,journals/bioinformatics/BerengerZSZ11,5.3230
1,MUMBO: MUlti-task Max-value Bayesian Optimizat...,journals/corr/abs-2006-12093,4.1820
2,Study of a Numerical Approach for a Transient ...,journals/na/ChakibGN03,4.1653
3,Fault-Tolerant Design Techniques for Semicondu...,journals/ibmrd/Aichelmann84,2.9362
4,The rise of accounting information systems.,journals/ijais/GrabskiL23,1.4433
5,DropFilterR: A Novel Regularization Method for...,journals/npl/PanNLSD20,1.3991
6,Multipath Aided Rapid Acquisition: Optimal Sea...,journals/tit/SuwansantisukW07,1.3347
7,Bordertown and the Globalisation of Justice: U...,journals/jilt/McInnes98,1.2845
8,A new combination of evidence based on comprom...,journals/fss/Yamada08,1.2808
9,Load dispersion-aware VM placement in favor of...,journals/tjs/NadjarAD17,1.2761


Given that data was created synthetically for the sole purpose of playing with the algorithm, we cannot extract much interpretation of the results obtained. With real life data, we could perhaps check if the results of the alleged most influential papers are really important papers in real life. 

However, it is clear that the pipeline was executed nicely and we obtained coherent results. 